In [104]:
import pandas as pd
import os
import numpy as np
import tkinter as tk
from IPython.display import display
from ipywidgets import Dropdown
from ipywidgets import DatePicker
from ipywidgets import Button, Layout
from scipy.stats import skew, mode
import xlsxwriter


# Переименовываем фалы в директориях
path = r"Total_by\\"
f = os.listdir(path)
n = 0
your_name = "BY"
format = '.xlsx'


for i in f:
    oldname = path + f[n]
    newname = path + your_name + str(n + 1) + format
    os.rename(oldname, newname)

    n += 1


path = r"Mounting\\"
f = os.listdir(path)
n = 0
your_name = "Монтажи"
format = '.xlsx'

for i in f:
    oldname = path + f[n]
    newname = path + your_name + str(n + 1) + format
    os.rename(oldname, newname)
    n += 1

# открывыем монтажи

df_mouting = mont_df = pd.read_excel('Mounting\\Монтажи1.xlsx', header=5, usecols=['ID поверхности', 'Дата монтажа'],
                                     thousands='.')

# Преобразвываем даты и пустые значения в 1 и 0
df_mouting_filled = df_mouting.fillna(1)
df_mouting_filled.loc[(mont_df['Дата монтажа'] == 'Пролонгировано'), 'Дата монтажа'] = 0
df_mouting_filled.loc[(mont_df['Дата монтажа'] == 'Монтаж не состоялся'), 'Дата монтажа'] = 0
df_mouting_filled_int = df_mouting_filled.astype(int)
df_mouting_filled_int.loc[(df_mouting_filled_int['Дата монтажа'] != 0), 'Дата монтажа'] = 1



# Считаем кол-во монтажей для каждой ID

мounting_pivot = pd.pivot_table(df_mouting_filled_int,
                                index=['ID поверхности'],
                                values=['Дата монтажа'],
                                aggfunc=np.sum,
                                fill_value=0).reset_index()

# Фрейм данных о монтаже
мounting = pd.DataFrame({
    'Порядковый номер': мounting_pivot['ID поверхности'],
    'Количество монтажей': мounting_pivot['Дата монтажа'],
    'Кол-во монтажей клиент': '',
    'Кол-во монтажей поставщик': '',
    'Монтажи закупка': '',
    'Монтажи продажа': ''
})
мounting.to_excel('отладка\\мounting.xlsx')

#открываем системные фреймы 
montn = pd.read_excel('data\\data.xlsx',sheet_name='month')
responsible = pd.read_excel('data\\data.xlsx',sheet_name='responsible')
agency = pd.read_excel('data\\data.xlsx',sheet_name='agency')
grouphead = pd.read_excel('data\\data.xlsx',sheet_name='grouphead')

#формируем листы для выпадающих списков
mont_filt = montn['month'].tolist()
responsible_filt = responsible['responsible'].tolist()
agency_filt = agency['agency'].tolist()
grouphead_filt = grouphead['grouphead'].tolist()

month_choice = []
responsible_choice = []
agency_choice = []
grouphead_choice = []

#Выбор ответсвенного
dropdown1 = Dropdown(
    options= responsible_filt,
    description='Баер:' 
        )

def on_button_clicked_month(b):
    responsible_choice.clear()
    responsible_choice.append(b['new'])

dropdown1.observe(on_button_clicked_month, names='value')
display(dropdown1)

#Выбор ГХ
dropdown1 = Dropdown(
    options= grouphead_filt,
    description='ГХ:' 
        )

def on_button_clicked_month(b):
    grouphead_choice.clear()
    grouphead_choice.append(b['new'])

dropdown1.observe(on_button_clicked_month, names='value')
display(dropdown1)

#Выбор агентсва
dropdown1 = Dropdown(
    options= agency_filt,
    description='Агент:' 
        )

def on_button_clicked_month(b):
    agency_choice.clear()
    agency_choice.append(b['new'])

dropdown1.observe(on_button_clicked_month, names='value')
display(dropdown1)


#Выбор Периода и клиента
dropdown1 = Dropdown(
    options= mont_filt,
    description='Месяц:' 
        )

def on_button_clicked_month(b):
    month_choice.clear()
    month_choice.append(b['new'])
    #Вибираем Клиента
    df_client = pd.read_excel('Total_by\\BY1.xlsx', sheet_name='Свод', header=2, usecols=['Клиент', month_choice[0]])
    df_client_filtr = df_client[df_client[month_choice[0]] != 0]  # Формируем список активных клиентов в периоде
    df_client_filtr_to_client = df_client_filtr['Клиент']  # Оставляем только лист с клиентами
    df_client_filtr_to_client.to_excel('data\\client.xlsx',index = False)    
    client = pd.read_excel('data\\client.xlsx')
    client1 = pd.read_excel('data\\client1.xlsx')
    client3 = pd.concat([client, client1])
    client_filt = client3['Клиент'].tolist()
    client_filt.sort()
    client_choice = []
    
    dropdown1 = Dropdown(
    options= client_filt,
    description='Клиент:' )

    def on_button_clicked(b):
        by = pd.read_excel('Total_by\\BY1.xlsx',sheet_name = b['new'],header = 1)
        by_df = pd.DataFrame(by)
        by_filtr = by_df.loc[by_df[month_choice[0]] == 1]
        client_choice.append(b['new'])
        by = by.fillna(0)
        by['ко-во поставщик январь'] = 0
        by['ко-во поставщик февраль'] = 0
        by['ко-во поставщик март'] = 0
        by['ко-во поставщик апрель'] = 0
        by['ко-во поставщик май'] = 0
        by['ко-во поставщик июнь'] = 0
        by['ко-во поставщик июль'] = 0
        by['ко-во поставщик август'] = 0
        by['ко-во поставщик сентябрь'] = 0
        by['ко-во поставщик октябрь'] = 0
        by['ко-во поставщик ноябрь'] = 0
        by['ко-во поставщик декабрь'] = 0
        by.loc[(by['закупка Январь net'] != 0),'ко-во поставщик январь'] = 1
        by.loc[(by['закупка Февраль net'] != 0),'ко-во поставщик февраль'] = 1
        by.loc[(by['закупка Март net'] != 0),'ко-во поставщик март'] = 1
        by.loc[(by['закупка Апрель net'] != 0),'ко-во поставщик апрель'] = 1
        by.loc[(by['закупка Май net'] != 0),'ко-во поставщик май'] = 1
        by.loc[(by['закупка Июнь net'] != 0),'ко-во поставщик июнь'] = 1
        by.loc[(by['закупка Июль net'] != 0),'ко-во поставщик июль'] = 1
        by.loc[(by['закупка Август net'] != 0),'ко-во поставщик август'] = 1
        by.loc[(by['закупка Сентябрь net'] != 0),'ко-во поставщик сентябрь'] = 1
        by.loc[(by['закупка Октябрь net'] != 0),'ко-во поставщик октябрь'] = 1
        by.loc[(by['закупка Ноябрь net'] != 0),'ко-во поставщик ноябрь'] = 1
        by.loc[(by['закупка Декабрь net'] != 0),'ко-во поставщик декабрь'] = 1
        
        by.to_excel('отладка\\by.xlsx',index = False)
                
        merged = pd.merge(by_filtr, мounting)
        merged.loc[ (merged['Стоимость первичного монтажа net']== 0),'Кол-во монтажей клиент'] = merged['Количество монтажей'] - 1
        merged.loc[ (merged['Стоимость первичного монтажа net'] != 0),'Кол-во монтажей клиент'] = merged['Количество монтажей']
        merged.loc[ (merged['Стоимость первичного монтажа net закупка']== 0),'Кол-во монтажей поставщик'] = merged['Количество монтажей'] - 1
        merged.loc[ (merged['Стоимость первичного монтажа net закупка'] != 0),'Кол-во монтажей поставщик'] = merged['Количество монтажей']        
        
        merged = merged.fillna(0)
        merged['Монтажи закупка'] = merged['Стоимость первичного монтажа net закупка'] + (merged['Количество монтажей']-1) * merged['Стоимость доп. монтажа net закупка']
        merged['Монтажи продажа'] = merged['Стоимость первичного монтажа net'] + (merged['Количество монтажей']-1) * merged['Стоимость доп. монтажа, net']
        merged['ко-во поставщик январь'] = 0
        merged['ко-во поставщик февраль'] = 0
        merged['ко-во поставщик март'] = 0
        merged['ко-во поставщик апрель'] = 0
        merged['ко-во поставщик май'] = 0
        merged['ко-во поставщик июнь'] = 0
        merged['ко-во поставщик июль'] = 0
        merged['ко-во поставщик август'] = 0
        merged['ко-во поставщик сентябрь'] = 0
        merged['ко-во поставщик октябрь'] = 0
        merged['ко-во поставщик ноябрь'] = 0
        merged['ко-во поставщик декабрь'] = 0
        merged.loc[ (merged['закупка Январь net'] != 0),'ко-во поставщик январь'] = 1
        merged.loc[ (merged['закупка Февраль net'] != 0),'ко-во поставщик февраль'] = 1
        merged.loc[ (merged['закупка Март net'] != 0),'ко-во поставщик март'] = 1
        merged.loc[ (merged['закупка Апрель net'] != 0),'ко-во поставщик апрель'] = 1
        merged.loc[ (merged['закупка Май net'] != 0),'ко-во поставщик май'] = 1
        merged.loc[ (merged['закупка Июнь net'] != 0),'ко-во поставщик июнь'] = 1
        merged.loc[ (merged['закупка Июль net'] != 0),'ко-во поставщик июль'] = 1
        merged.loc[ (merged['закупка Август net'] != 0),'ко-во поставщик август'] = 1
        merged.loc[ (merged['закупка Сентябрь net'] != 0),'ко-во поставщик сентябрь'] = 1
        merged.loc[ (merged['закупка Октябрь net'] != 0),'ко-во поставщик октябрь'] = 1
        merged.loc[ (merged['закупка Ноябрь net'] != 0),'ко-во поставщик ноябрь'] = 1
        merged.loc[ (merged['закупка Декабрь net'] != 0),'ко-во поставщик декабрь'] = 1
        
        merged.to_excel('отладка\\storage.xlsx',index = False)
        
        # Создаем кнопку для формироания закупки
        button1 = Button(description="Сделать закупку!", 
                        button_style='success',
                        layout=Layout(width='30%', height='80px'))

        def on_button_clicked(b): # Описываем обработчик события
            #Формируем листы для формирования сводных таблиц
                        
            storage = pd.read_excel('отладка\\storage.xlsx')
            if len(storage) != 0:
                
                storage_colums = storage.columns
           
                storage_month = storage_colums[19:32]
                storage_placement_sale = storage_colums[33:46]
                storage_placement_purchase = storage_colums [66:79]
                storage_placement_quantity = storage_colums [89:101]
                       
                #Формируем зависимость от списка
                for i in range (0,13):
                    if mont_filt[i] in month_choice:
                        sale_price = storage_placement_sale[i]
                        purchase_price = storage_placement_purchase[i]
                        quantity_client = storage_month[i]
                        quantity_supplier = storage_placement_quantity[i]
                       
                storage_pivot = pd.pivot_table(storage,
                                           index=['Город','Тип конструкции','Размер','Поставщик','Налог на размещение','Кампания','СК %','Налог на размещение закупка'],
                                           values=[sale_price,quantity_client,purchase_price,quantity_supplier],
                                           aggfunc=np.sum,
                                           fill_value=0).reset_index()
                        
                len_data = (len(storage_pivot['Город']))
                storage_pivot.reset_index()
                storage_pivot.to_excel('отладка\\storage_pivot.xlsx')
           
                placment_data =pd.DataFrame ({'Dax':' ', 
                    'город':storage_pivot['Город'],
                    'Вид медиа':'outdoor',
                    'Услуга': 'Размещение',
                    'Расположение/Носитель':storage_pivot['Тип конструкции'],
                    'Формат': storage_pivot['Размер'],
                    'Кол-во' : storage_pivot[quantity_client],
                    'Отв.': [responsible_choice[0] for i in range (len_data)],
                    'GH': [grouphead_choice[0] for i in range (len_data)],
                    'Агентство':[agency_choice[0] for i in range (len_data)],
                    'Клиент':[client_choice[0] for i in range (len_data)],
                    'Тип кампании': storage_pivot['Кампания'],
                    'бренд':[client_choice[0] for i in range (len_data)],
                    'С': '1',
                    'ПО':'31',
                    'Месяц':[month_choice[0] for i in range (len_data)],
                    'Клиент Налог': storage_pivot['Налог на размещение'],
                    ' ': storage_pivot[quantity_client]-storage_pivot[quantity_supplier],
                    'За единицу GROSS Клиент':' ',
                    'Клиент ИТОГО GROSS' :'',
                    'Клиент За единицу NET': '',
                    'Итого NET Клиент': storage_pivot[sale_price],
                    'Тип договора':' ',
                    'Подрячк Налог':storage_pivot['Налог на размещение закупка'],
                    'Подрядчик':storage_pivot['Поставщик'],
                    'кол-во Подрячик': storage_pivot[quantity_supplier],
                    'Подрячик За единицу GROSS' : ' ',
                    'ИТОГО GROSS Подрячик':' ',
                    'Подрячик За единицу  NET' : '',
                    'ИТОГО NET подрядчик' : storage_pivot[purchase_price],
                    'СК %': storage_pivot['СК %'],
                    'СК руб.': storage_pivot[sale_price] * storage_pivot['СК %'],
                    '% net':1-storage_pivot[purchase_price]/storage_pivot[sale_price],
                    'КC NET': storage_pivot[sale_price]-storage_pivot[purchase_price],
                    'Комментарии':'',
                     'ТЕХ': storage_pivot[purchase_price] + storage_pivot[sale_price]
                     })
                
                placment_data.to_excel('отладка\\placment_data.xlsx')
                                
                montage_pivot = pd.pivot_table(storage,
                                           index=['Город','Тип конструкции','Размер','Поставщик','Кампания','Монтажи закупка','Монтажи продажа','Налог на Монтаж закупка','Налог на монтаж'],
                                           values=['Кол-во монтажей клиент','Кол-во монтажей поставщик'],
                                           aggfunc=np.sum,
                                           fill_value=0).reset_index()
                                
                montage_pivot.to_excel('отладка\\montage_pivot.xlsx')
                
                montage_len_data = (len(montage_pivot['Город']))
                
                montage_data =pd.DataFrame ({'Dax':' ', 
                    'город': montage_pivot['Город'],
                    'Вид медиа':'outdoor',
                    'Услуга': 'монтаж',
                    'Расположение/Носитель':montage_pivot['Тип конструкции'],
                    'Формат': montage_pivot['Размер'],
                    'Кол-во' : montage_pivot['Кол-во монтажей клиент'],
                    'Отв.': [responsible_choice[0] for i in range (montage_len_data)],
                    'GH': [grouphead_choice[0] for i in range (montage_len_data)],
                    'Агентство':[agency_choice[0] for i in range (montage_len_data)],
                    'Клиент':[client_choice[0] for i in range (montage_len_data)],
                    'Тип кампании': montage_pivot['Кампания'],
                    'бренд':[client_choice[0] for i in range (montage_len_data)],
                    'С': '1',
                    'ПО':'31',
                    'Месяц':[month_choice[0] for i in range (montage_len_data)],
                    'Клиент Налог': montage_pivot['Налог на монтаж'],
                    ' ': montage_pivot['Кол-во монтажей поставщик']-montage_pivot['Кол-во монтажей клиент'],
                    'За единицу GROSS Клиент':' ',
                    'Клиент ИТОГО GROSS' :'',
                    'Клиент За единицу NET': '',
                    'Итого NET Клиент': montage_pivot['Монтажи продажа'],
                    'Тип договора':' ',
                    'Подрячк Налог':montage_pivot['Налог на Монтаж закупка'],
                    'Подрядчик':montage_pivot['Поставщик'],
                    'кол-во Подрячик': montage_pivot['Кол-во монтажей поставщик'],
                    'Подрячик За единицу GROSS' : ' ',
                    'ИТОГО GROSS Подрячик':' ',
                    'Подрячик За единицу  NET' : '',
                    'ИТОГО NET подрядчик' : montage_pivot['Монтажи закупка'],
                    'СК %':'0',
                    'СК руб.': '0',
                    '% net': 1-montage_pivot['Монтажи закупка']/montage_pivot['Монтажи продажа'],
                    'КC NET':montage_pivot['Монтажи продажа'] - montage_pivot['Монтажи закупка'],
                    'Комментарии':'',
                     'ТЕХ': montage_pivot ['Монтажи закупка'] + montage_pivot['Монтажи продажа']
                     })
                
                
                montage_data.to_excel('отладка\\montage_data.xlsx')
                
                placment_data_montage_data = pd.concat([placment_data,montage_data])
                placment_data_montage_data_copy = placment_data_montage_data.copy(deep=True)
                placment_data_montage_data_copy.to_excel('отладка\\placment_data_montage_data_copy.xlsx')
                
                placment_data_montage_data_copy = placment_data_montage_data_copy.loc[placment_data_montage_data_copy['ТЕХ'] != 0]
                placment_data_montage_data_copy.pop('ТЕХ')
                placment_data_montage_data_copy.to_excel('Закупки\\закупка.xlsx',index = False)
                
                
            else:
                
                by_storage = pd.read_excel('отладка\\by.xlsx')
                by_colums = by.columns
                by_month = by_colums [19:32]
                by_placement_sale = by_colums [33:46]
                by_placement_purchase = by_colums [66:79]
                by_storage_placement_quantity = by_colums [88:101]
                
                
                for i in range (0,13):
                    if mont_filt[i] in month_choice:
                        by_sale_price = by_placement_sale[i]
                        by_purchase_price = by_placement_purchase[i]
                        by_quantity_client = by_month[i]
                        by_quantity_supplier = by_storage_placement_quantity[i]
                                
                by_storage_pivot = pd.pivot_table(by_storage,
                                           index=['Город','Тип конструкции','Размер','Поставщик','Налог на размещение','Кампания','СК %','Налог на размещение закупка'],
                                           values=[by_sale_price,by_quantity_client,by_purchase_price,by_quantity_supplier],
                                           aggfunc=np.sum,
                                           fill_value=0).reset_index()
                by_storage_pivot.to_excel('отладка\\by_storage_pivot.xlsx')
                
                
                by_len_data = (len(by_storage_pivot['Город']))
                by_storage_pivot.reset_index()
                        
                
                by_placment_data =pd.DataFrame ({'Dax':' ', 
                    'город':by_storage_pivot['Город'],
                    'Вид медиа':'outdoor',
                    'Услуга': 'Размещение',
                    'Расположение/Носитель':by_storage_pivot['Тип конструкции'],
                    'Формат': by_storage_pivot['Размер'],
                    'Кол-во' : by_storage_pivot[by_quantity_client],
                    'Отв.': [responsible_choice[0] for i in range (by_len_data)],
                    'GH': [grouphead_choice[0] for i in range (by_len_data)],
                    'Агентство':[agency_choice[0] for i in range (by_len_data)],
                    'Клиент':[client_choice[0] for i in range (by_len_data)],
                    'Тип кампании': by_storage_pivot['Кампания'],
                    'бренд':[client_choice[0] for i in range (by_len_data)],
                    'С': '1',
                    'ПО':'31',
                    'Месяц':[month_choice[0] for i in range (by_len_data)],
                    'Клиент Налог': by_storage_pivot['Налог на размещение'],
                    ' ': by_storage_pivot[by_quantity_supplier]-by_storage_pivot[by_quantity_client],
                    'За единицу GROSS Клиент':' ',
                    'Клиент ИТОГО GROSS' :'',
                    'Клиент За единицу NET': '',
                    'Итого NET Клиент': by_storage_pivot[by_sale_price],
                    'Тип договора':' ',
                    'Подрячк Налог':by_storage_pivot['Налог на размещение закупка'],
                    'Подрядчик':by_storage_pivot['Поставщик'],
                    'кол-во Подрячик': by_storage_pivot[by_quantity_supplier],
                    'Подрячик За единицу GROSS' : ' ',
                    'ИТОГО GROSS Подрячик':' ',
                    'Подрячик За единицу  NET' : '',
                    'ИТОГО NET подрядчик' : by_storage_pivot[by_purchase_price],
                    'СК %': by_storage_pivot['СК %'],
                    'СК руб.': by_storage_pivot[by_sale_price] * by_storage_pivot['СК %'],
                    '% net':1-by_storage_pivot[by_purchase_price]/by_storage_pivot[by_sale_price],
                    'КC NET': by_storage_pivot[by_sale_price] - by_storage_pivot[by_purchase_price],
                    'Комментарии':' ',
                     'ТЕХ': by_storage_pivot [by_sale_price] + by_storage_pivot[by_purchase_price]
                     })
                
                by_placment_data_copy = by_placment_data.copy(deep=True)
                by_placment_data_copy =  by_placment_data_copy.loc[ by_placment_data_copy['ТЕХ'] != 0]
                by_placment_data_copy.pop('ТЕХ')
                by_placment_data_copy.to_excel('Закупки\\закупка.xlsx',index = False)
                
            purchase_of_replacement = pd.read_excel('Закупки\\закупка.xlsx')
            
            zakupka = purchase_of_replacement.replace({'Клиент Налог':{'НДС':1.2, 'ндс': 1,'Ндс': 1,
                          'БЕЗ НДС':1, 'Без НДС': 1},
                          'Подрячк Налог':{'НДС':1.2, 'ндс':1.2,'Ндс':1.2,'БЕЗ НДС':1, 'Без НДС': 1},
                          'Расположение/Носитель':{'Billboard 3x6':'Щиты','SuperSite':'Щиты',
                          'Cityboard':'Щиты', 'Digital Billboard 3х6': 'Щиты цифровые',
                          'Остановка':'Остановки','Digital Cityboard':'Щиты цифровые',
                          'SS Digital':'Щиты цифровые','Cityformat':'Щиты'},
                          'Подрядчик':{'Russ Outdoor':'РУСС', 'Gallery':'Гэллери',
                          'АО 21 Век ТВ': '21 Век АО ','Восток Медиа':'Восток-Медиа',
                          'Sunlight Outdoor':'Санлайт','GMG':'ГМГ' },
                          'Формат':{'3.7х2.7':'2.7х3.7'},
                          'Месяц': {'Январь':'1/ЯНВАРЬ/22','Февраль':'2/ФЕВРАЛЬ/22','Март':'3/МАРТ/22',
                                   'Апрель':'4/АПРЕЛЬ/22','Май':'5/МАЙ/22','Июнь':'6/ИЮНЬ/22'
                                   ,'Июль':'7/ИЮЛЬ/22','Август':'8/АВГУСТ/22','Сентябрь':'9/СЕНТЯБРЬ/22',
                                   'Октябрь':'10/ОКТЯБРЬ/22','Ноябрь':'11/НОЯБРЬ/22','Декабрь':'12/ДЕКАБРЬ/22'}
                         })
            
            zakupka['Клиент За единицу NET'] = zakupka['Итого NET Клиент'] / zakupka['Кол-во']
            zakupka['Клиент ИТОГО GROSS'] = zakupka['Итого NET Клиент'] * zakupka['Клиент Налог']
            zakupka['За единицу GROSS Клиент'] = zakupka['Клиент ИТОГО GROSS'] / zakupka['Кол-во']
            
            zakupka['ИТОГО GROSS Подрячик'] = zakupka['ИТОГО NET подрядчик'] * zakupka['Подрячк Налог']
            zakupka['Подрячик За единицу  NET'] = zakupka['ИТОГО NET подрядчик'] / zakupka['кол-во Подрячик']
            zakupka['Подрячик За единицу GROSS'] = zakupka['ИТОГО GROSS Подрячик'] / zakupka['кол-во Подрячик']
             
            zakupka.to_excel('Закупки\\закупка.xlsx',index = False)
                       
            writer = pd.ExcelWriter('Закупки\\закупка.xlsx', engine='xlsxwriter')
            zakupka.to_excel(writer, sheet_name='Закупка',
            startrow=2, startcol=0, index=False)  
            writer.close()
            
            name = client_choice[0] +' ' + month_choice[0] + ' ' + 'Закупка'+'.xlsx'
            os.rename('Закупки\\Закупка.xlsx', 'Закупки\\'+  name)
            
            print(f'Закупка по клиенту {client_choice[0]} за {month_choice[0]} готова')
            
        button1.on_click(on_button_clicked) 
        display(button1)
                    
    dropdown1.observe(on_button_clicked, names='value')   
    display(dropdown1)
    
           
dropdown1.observe(on_button_clicked_month, names='value')
display(dropdown1)
    


Dropdown(description='Баер:', options=(0, 'Акимова', 'Антонова', 'Астафьева', 'Башук', 'Винцковский', 'Гаврюши…

Dropdown(description='ГХ:', options=(0, 'Гусева', 'Докучаева', 'Долгов', 'Заманова', 'Зубач', 'Зюба', 'Ионов',…

Dropdown(description='Агент:', options=(0, 'OMD', 'VOLT', 'Innocean', 'Media Wise', 'PHD', 'Proximity', 'MADE'…

Dropdown(description='Месяц:', options=(0, 'Январь', 'Февраль', 'Март', 'Апрель', 'Май', 'Июнь', 'Июль', 'Авгу…

Dropdown(description='Клиент:', options=('!', 'OBI', 'SCJ', 'Samsung', 'Stada', 'VCR', 'Виктория', 'Леруа', 'О…

Button(button_style='success', description='Сделать закупку!', layout=Layout(height='80px', width='30%'), styl…

Закупка по клиенту Леруа за Март готова
